# Memory

调用大模型API，仅仅是输入-> 输出，根据输入数据，推理生成输出数据，这是无状态的，所以大模型也没有所谓的记忆。我们应用大模型时实现的记忆，本质是把前面的对话情况，也作为本轮对话的输出数据，提供给LLM，这样 LLM 就知道之前的对话情况，外部看起来就是有了记忆。

直接调用 LLM 的 API 时，想让 LLM 有记忆，需要手动把之前的对话组织好，一并传给 LLM；使用 Langchain 时，可以直接调用 ConversationChain，并提供记忆的保存策略。记忆保存策略主要有：

- ConversationBufferMemory：不对前面的多轮对话做限制
- ConversationBufferWindowMemory：通过滑动窗口，限制前面多轮对话的轮数，仅保留最新 k 轮对话
- ConversationSummaryMemory：每轮对话后，通过 LLM 做一次总结，再放置到记忆中；这个策略比较费钱
- ConversationSummaryBufferMemory：结合了滑动窗口和总结，前面对话轮数超过 k 轮时，再将这些对话总结为一轮，放入记忆。


# ConversationChain

In [3]:
from langchain_openai import AzureChatOpenAI
# from langchain.chains import ConversationChain
from langchain.chains.conversation.base import ConversationChain

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=512
)

conv_chain = ConversationChain(llm=llm)
print(conv_chain.prompt)

input_variables=['history', 'input'] template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'


# BufferMemory

In [4]:
from langchain_openai import AzureChatOpenAI
from langchain.chains.conversation.base import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens = 128
)

conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())

In [5]:
response = conversation.invoke("我姐姐明天要过生日，我需要一束生日花束。")
print("第一次对话后的记忆:", conversation.memory.buffer)
print("1.response: ", response)

# 回合2
response = conversation.invoke("她喜欢粉色玫瑰，颜色是粉色的。")
print("第二次对话后的记忆:", conversation.memory.buffer)
print("2.response: ", response)

# 回合3 （第二天的对话）
response = conversation.invoke("我又来了，还记得我昨天为什么要来买花吗？")
print("/n第三次对话后时提示:/n",conversation.prompt.template)
print("/n第三次对话后的记忆:/n", conversation.memory.buffer)
print("3.response: ", response)

第一次对话后的记忆: Human: 我姐姐明天要过生日，我需要一束生日花束。
AI: 那真是太好了！为你姐姐准备生日花束一定能为她的生日增添不少色彩和温馨。通常，生日花束可以包含一些象征美好祝愿的花朵。以下是几种常见的选择：

1. **玫瑰花**：红玫瑰象征爱情和尊重，粉玫瑰象征感激和钦佩，白玫瑰象征纯洁和新开始。
2. **百合花**：象征纯洁和高雅，白色百合尤为适
1.response:  {'input': '我姐姐明天要过生日，我需要一束生日花束。', 'history': '', 'response': '那真是太好了！为你姐姐准备生日花束一定能为她的生日增添不少色彩和温馨。通常，生日花束可以包含一些象征美好祝愿的花朵。以下是几种常见的选择：\n\n1. **玫瑰花**：红玫瑰象征爱情和尊重，粉玫瑰象征感激和钦佩，白玫瑰象征纯洁和新开始。\n2. **百合花**：象征纯洁和高雅，白色百合尤为适'}


/Users/gevin/projects/pycharm/ai/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


第二次对话后的记忆: Human: 我姐姐明天要过生日，我需要一束生日花束。
AI: 那真是太好了！为你姐姐准备生日花束一定能为她的生日增添不少色彩和温馨。通常，生日花束可以包含一些象征美好祝愿的花朵。以下是几种常见的选择：

1. **玫瑰花**：红玫瑰象征爱情和尊重，粉玫瑰象征感激和钦佩，白玫瑰象征纯洁和新开始。
2. **百合花**：象征纯洁和高雅，白色百合尤为适
Human: 她喜欢粉色玫瑰，颜色是粉色的。
AI: 那简直太完美了！粉色玫瑰既美丽又充满了温柔和感激的意味，非常适合作为生日花束。你可以选择以下几种搭配，让花束更加丰富和有层次感：

1. **粉色玫瑰**：这是花束的主花，可以选择不同的粉色深浅来增加视觉效果。
2. **满天星**：这些小小的白色花朵可以作为填充花，让整个花束显得更加饱满和雅致。
3. **粉色康乃
2.response:  {'input': '她喜欢粉色玫瑰，颜色是粉色的。', 'history': 'Human: 我姐姐明天要过生日，我需要一束生日花束。\nAI: 那真是太好了！为你姐姐准备生日花束一定能为她的生日增添不少色彩和温馨。通常，生日花束可以包含一些象征美好祝愿的花朵。以下是几种常见的选择：\n\n1. **玫瑰花**：红玫瑰象征爱情和尊重，粉玫瑰象征感激和钦佩，白玫瑰象征纯洁和新开始。\n2. **百合花**：象征纯洁和高雅，白色百合尤为适', 'response': '那简直太完美了！粉色玫瑰既美丽又充满了温柔和感激的意味，非常适合作为生日花束。你可以选择以下几种搭配，让花束更加丰富和有层次感：\n\n1. **粉色玫瑰**：这是花束的主花，可以选择不同的粉色深浅来增加视觉效果。\n2. **满天星**：这些小小的白色花朵可以作为填充花，让整个花束显得更加饱满和雅致。\n3. **粉色康乃'}
/n第三次对话后时提示:/n The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it

减少 token 数，看有何区别。
可以发现，记忆还是有的，但是对话信息量不足，影响持续对话的质量

In [7]:
llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=32
)

conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())
response = conversation.invoke("我姐姐明天要过生日，我需要一束生日花束。")
print("第一次对话后的记忆:", conversation.memory.buffer)
print("1.response: ", response['response'])

# 回合2
response = conversation.invoke("她喜欢粉色玫瑰，颜色是粉色的。")
print("第二次对话后的记忆:", conversation.memory.buffer)
print("2.response: ", response.get("response"))

# 回合3 （第二天的对话）
response = conversation.invoke("我又来了，还记得我昨天为什么要来买花吗？")
# print("/n第三次对话后时提示:/n", conversation.prompt.template)
print("/n第三次对话后的记忆:/n", conversation.memory.buffer)
print("3.response: ", response.get("response"))

第一次对话后的记忆: Human: 我姐姐明天要过生日，我需要一束生日花束。
AI: 那是个好主意！生日花束可以让人特别开心。你知道你姐姐喜欢什么花吗？如果不确定，我可以给你
1.response:  那是个好主意！生日花束可以让人特别开心。你知道你姐姐喜欢什么花吗？如果不确定，我可以给你
第二次对话后的记忆: Human: 我姐姐明天要过生日，我需要一束生日花束。
AI: 那是个好主意！生日花束可以让人特别开心。你知道你姐姐喜欢什么花吗？如果不确定，我可以给你
Human: 她喜欢粉色玫瑰，颜色是粉色的。
AI: 粉色玫瑰真是一个绝佳的选择，既典雅又充满温柔的气息。你可以选择一个以粉
2.response:  粉色玫瑰真是一个绝佳的选择，既典雅又充满温柔的气息。你可以选择一个以粉
/n第三次对话后的记忆:/n Human: 我姐姐明天要过生日，我需要一束生日花束。
AI: 那是个好主意！生日花束可以让人特别开心。你知道你姐姐喜欢什么花吗？如果不确定，我可以给你
Human: 她喜欢粉色玫瑰，颜色是粉色的。
AI: 粉色玫瑰真是一个绝佳的选择，既典雅又充满温柔的气息。你可以选择一个以粉
Human: 我又来了，还记得我昨天为什么要来买花吗？
AI: 当然记得！你昨天提到你姐姐明天要过生日，你需要一束粉色玫瑰的生日花束。你还有
3.response:  当然记得！你昨天提到你姐姐明天要过生日，你需要一束粉色玫瑰的生日花束。你还有


# BufferWindowMemory

In [17]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=512
)

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)

# 第一天的对话
# 回合1
result = conversation.invoke("我姐姐明天要过生日，我需要一束生日花束。")
print(result.get('response'))

# 回合 2，此时问它是记得的
result = conversation.invoke("姐姐什么时候过生日？")
print("\n第二次的Response: ", result.get("response"))
print("\n第二次的记忆：\n", conversation.memory.buffer)

# 回合 3
result = conversation.invoke("她喜欢粉色玫瑰，颜色是粉色的。")
print("\n第三次对话后的记忆:\n", conversation.memory.buffer)
print(result)

# 回合 4
# 由于k=1只保存一轮对话，此时已经不记得了
result = conversation.invoke("姐姐什么时候过生日？")
print("\n第四次的Response: ", result.get("response"))
print("\n第四次的记忆：\n", conversation.memory.buffer)


那真是太好了！生日花束是个非常特别的礼物呢。你有想过要送什么类型的花束吗？很多人会选择玫瑰、百合或者郁金香这些经典的花卉。玫瑰象征爱情、尊敬和友谊，百合代表纯洁和高贵，而郁金香则象征美丽和幸福。如果你姐姐有特别喜欢的花，那就更好了！你也可以根据花语来挑选适合她的花束。如果你需要一些推荐或者帮助，我很乐意提供更多的建议哦！

第二次的Response:  你刚才提到你姐姐的生日是明天，所以她的生日应该是明天哦！你现在可以开始准备生日花束和其他庆祝活动，相信她一定会非常开心的。如果你需要更多关于花束挑选的建议或者其他生日策划的点子，请随时告诉我！

第二次的记忆：
 Human: 姐姐什么时候过生日？
AI: 你刚才提到你姐姐的生日是明天，所以她的生日应该是明天哦！你现在可以开始准备生日花束和其他庆祝活动，相信她一定会非常开心的。如果你需要更多关于花束挑选的建议或者其他生日策划的点子，请随时告诉我！

第三次对话后的记忆:
 Human: 她喜欢粉色玫瑰，颜色是粉色的。
AI: 那真是太好了！粉色玫瑰象征着优雅、温柔和感激之情，非常适合作为生日礼物。你可以考虑以下几种方式来搭配粉色玫瑰，打造一个完美的生日花束：

1. **粉色玫瑰单独成束**：简单而经典，可以选择11朵、19朵或是33朵这样的吉利数字。

2. **混合花束**：可以搭配一些白色满天星、粉色康乃馨或是紫色洋桔梗，这些花朵和粉色玫瑰搭配起来会显得非常柔美和浪漫。

3. **花篮**：如果你想要一个更独特的方式，可以把粉色玫瑰和其他鲜花一起放入一个漂亮的花篮里，既有观赏性又能长时间保存。

另外，你还可以添加一些装饰，比如精美的缎带、蝴蝶结或者一些小巧的装饰品，让花束看起来更精致。另外别忘了附上一张亲手写的生日卡片，表达你的祝福和心意。

希望这些建议对你有帮助，祝你姐姐生日快乐！如果你需要更多的建议或者有其他问题，随时告诉我哦！
{'input': '她喜欢粉色玫瑰，颜色是粉色的。', 'history': 'Human: 姐姐什么时候过生日？\nAI: 你刚才提到你姐姐的生日是明天，所以她的生日应该是明天哦！你现在可以开始准备生日花束和其他庆祝活动，相信她一定会非常开心的。如果你需要更多关于花束挑选的建议或者其他生日策划的点子，请随时告诉我！', 'response': '那真是太好了！粉色玫瑰象征

# Summary Memory

In [19]:
# from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.memory import ConversationSummaryMemory

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=512
)

conversation = ConversationChain(
    llm = llm,
    memory=ConversationSummaryMemory(llm=llm)
)

# 回合 1
result = conversation.invoke("我姐姐明天要过生日，我需要一束生日花束。")
print("\n第一次的Response: \n", result.get('response'))

# 回合 2
result = conversation.invoke("姐姐什么时候过生日？")
print("\n第二次的Response: \n", result.get("response"))


# 回合 3
result = conversation.invoke("她喜欢粉色玫瑰，颜色是粉色的。")
print("\n第三次的Response: \n", result)

# 回合 4
result = conversation.invoke("姐姐什么时候过生日？")
print("\n第四次的Response: ", result.get("response"))



第一次的Response: 
 那真是太好了！为你姐姐准备生日花束是个很贴心的想法。你有具体的花卉偏好吗？比如说，她喜欢玫瑰、百合、康乃馨或者是向日葵之类的花吗？如果你不确定的话，我可以推荐一些常见的生日花束组合。

1. **玫瑰和百合的组合**：这是一个经典的组合，玫瑰象征着爱和美丽，而百合象征纯洁和高贵。
2. **向日葵和满天星的组合**：向日葵代表着阳光和快乐，而满天星则给花束增添了一种梦幻的感觉。
3. **康乃馨和郁金香的组合**：康乃馨代表着感情和喜悦，郁金香则象征着优雅和繁荣。

你可以根据你姐姐的喜好和个性来选择合适的花束。如果你还有其他具体的要求或者问题，也可以告诉我，我很乐意帮忙！

第二次的Response: 
 你提到你姐姐的生日是明天，所以看来她的生日是10月14日。你已经考虑给她送一束花作为生日礼物，这是个非常贴心的想法！如果你有任何关于花束的具体要求或需要进一步的建议，请随时告诉我。

第三次的Response: 
 {'input': '她喜欢粉色玫瑰，颜色是粉色的。', 'history': "The human mentions that their sister's birthday is tomorrow and they need a birthday bouquet. The AI responds positively, asking if the human has specific floral preferences such as roses, lilies, carnations, or sunflowers, and offers to recommend common birthday bouquet combinations if the human is unsure. The AI then suggests three combinations: roses and lilies for love and purity, sunflowers and baby's breath for sunshine and whimsy, and carnations and tulips for affection and elegance. The AI invites the human to sh

# SummaryBuffer Memory

In [22]:
from langchain.memory import ConversationSummaryBufferMemory

llm = AzureChatOpenAI(
    model_name="gpt-4o",
    temperature=0.8,
    max_tokens=512
)

# 即使记忆存储比较小，也能记住
conversation = ConversationChain(
    llm = llm,
    memory=ConversationSummaryBufferMemory(max_token_limit=64, llm=llm)
)

# 回合 1
result = conversation.invoke("我姐姐5月10日要过生日，我需要一束生日花束。")
print("\n第一次的Response: \n", result.get('response'))

# 回合 2
result = conversation.invoke("姐姐什么时候过生日？")
print("\n第二次的Response: \n", result.get("response"))


# 回合 3
result = conversation.invoke("她喜欢粉色玫瑰，颜色是粉色的。")
print("\n第三次的Response: \n", result)

# 回合 4
result = conversation.invoke("姐姐什么时候过生日？")
print("\n第四次的Response: ", result.get("response"))


第一次的Response: 
 那太好了！为你姐姐准备一束生日花束是个美好的主意。5月是春季，许多花儿正值盛开季节，选择非常多样化。以下是一些常见且适合作为生日花束的花卉：

1. **玫瑰**：玫瑰花象征爱与美，是生日花束的经典选择。你可以选择红色玫瑰代表爱意，或粉色玫瑰代表温柔。

2. **郁金香**：郁金香色彩丰富，代表高雅和美丽，非常适合春季生日。

3. **百合**：百合花香气浓郁，象征纯洁与高贵。白色百合非常适合作为生日花束。

4. **康乃馨**：康乃馨象征爱和关怀，适合作为亲人之间的生日花束。粉色和白色康乃馨都很适合。

5. **向日葵**：向日葵充满阳光与活力，象征着积极和快乐，能给生日增添一份明亮的色彩。

6. **牡丹**：牡丹花在5月正是盛开时期，象征富贵和美丽，是非常适合的选择。

你可以选择单一品种的花束，也可以混合搭配几种花卉，让花束更加丰富和有层次感。同时，别忘了搭配一些绿叶植物，如尤加利叶或满天星，以增强花束的整体美观度。

如果你有任何具体的花店推荐或价格预算，我可以进一步提供建议哦！

第二次的Response: 
 你姐姐的生日是5月10日。你刚刚提到的。你在考虑给她准备生日花束，我觉得这真是个不错的主意！如果你需要更多关于花束的建议或者有特定的花店推荐和预算限制，可以告诉我哦，我很乐意帮忙。

第三次的Response: 
 {'input': '她喜欢粉色玫瑰，颜色是粉色的。', 'history': "System: The human mentions that their sister's birthday is on May 10th and they need a bouquet of birthday flowers. The AI suggests that preparing a birthday bouquet is a wonderful idea and provides a list of flowers that are suitable for a spring birthday, including roses, tulips, lilies, carnations, sunflowers, and peonies. The AI also advises on mixing di